In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 10 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

26 December 2022 ; 14:21:25


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,12,0,5.8,1.1,2.0,0.542,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,1,1
1,Bobby Portis,PF,27,MIL,33,8,26.0,5.7,11.5,0.500,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,0,1
3,George Hill,PG,36,MIL,29,0,19.8,1.6,3.8,0.427,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,0,1
7,Joe Ingles,SF,35,MIL,4,0,17.3,1.0,4.8,0.211,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,0,1
8,Jordan Nwora,SF,24,MIL,26,3,16.6,2.0,5.5,0.368,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,0,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,23.6,29,0,BOS,18.3,2.262069,5.655172,13.6,1,2.6,3.675862,0.283647
1,Bobby Portis,26.0,33,8,MIL,18.1,1.490909,6.212121,13.4,2,1.4,2.981818,0.136477
2,Bones Hyland,20.4,24,0,DEN,14.7,0.341667,1.366667,13.0,1,0.9,0.341667,0.116651
3,Norman Powell,24.9,24,5,LAC,15.1,0.683333,3.416667,15.0,4,-0.6,1.025000,0.109283
4,Grant Williams,28.0,33,14,BOS,12.4,1.490909,6.212121,9.2,1,-0.2,3.975758,0.087923


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)